# <center> Gesture Recognition Project

## Table of Content

1. [Environment Setup in Anaconda3](#Environment-Setup-in-Anaconda3)
2. [Problem Statement](#Problem-Statement)
3. [Importing all the important libraries](#Importing-all-the-important-libraries)  
4. [Generator](#Generator)
5. [Model](#Model)
    1. [Model 1 and 2 - Conv3D](#Model-1-and-2---Conv3D)
    2. [Model 3 - Conv3D](#Model-3---Conv3D)
    3. [Model 4 and 5 - Conv3D](#Model-4-and-5---Conv3D)
    4. [Models 6 and 7 - Conv3D](#Models-6-and-7---Conv3D)
    5. [Model 8 - Conv3D](#Model-8---Conv3D)
    6. [Model 9 - TimeDistributed Conv2D + GRU](#Model----TimeDistributed-Conv2D-+-GRU)
    7. [Model 10 - TimeDistributed Conv2D + GRU](#Model-10---TimeDistributed-Conv2D-+-GRU)
    8. [Model 11 - TimeDistributed Conv2D + Dense](#Model-11---TimeDistributed-Conv2D-+-Dense)
6. [FINAL MODEL - Time Distributed + ConvLSTM2D](#FINAL-MODEL---Time-Distributed-+-ConvLSTM2D)

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly.

* Jheser Guzman

## Environment Setup in Anaconda
* Python 3.7.13
* TensorFlow 2.0.0
* Keras 2.3.1

## Problem Statement

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command

## Importing all the important libraries

In [18]:
# import important libraries
import numpy as np
import pandas as pd
import os

from skimage.io import imread
from skimage.transform import resize as imresize
from PIL import Image, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [20]:
import os
np.random.seed(30)
import random as rn
rn.seed(30)

# import ML libraries
import tensorflow as tf
from keras import backend as K

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [21]:
# train and validatiaon data creator with ablation
# when ablation is 10, then 10 folders for each label will be selected.

def get_data(path, ablation=None):
    train_doc = np.random.permutation(open(path + 'train.csv').readlines())
    val_doc = np.random.permutation(open(path + 'val.csv').readlines())
    counts = np.zeros(5) # count for loading folders for 5 classes
    train_data = []
    val_data = []

    # when ablation is None pass full training and val data
    if ablation is not None:
        
        # iterating train doc
        for doc in train_doc:
            lable = int(doc.strip().split(';')[2])
            if counts[lable] < ablation:
                train_data.append(doc)
                counts[lable] += 1 
        counts = np.zeros(5)
        
        # iterating val doc
        for doc in val_doc:
            lable = int(doc.strip().split(';')[2])
            if counts[lable] < ablation:
                val_data.append(doc)
                counts[lable] += 1
    else:
        train_data, val_data = train_doc, val_doc
    return train_data, val_data

In [22]:
train_doc, val_doc = get_data('_data/', ablation=None)
batch_size = 32
enable_augmentation = False

# selecting alternate frames from 7 to 26.
seq_idx = range(7,26,2)

# image dimensions
dim_x, dim_y = 120, 120

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
# Generator with augmentation for train data
def generator(source_path, folder_list, batch_size, is_train = False, augmention = False, debug=False):
    # print('\nSource path = ', source_path, '; batch size =', batch_size)
    img_idx = seq_idx #create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y, z = dim_x, dim_y
    while True:
        # doubling the data for augmentation
        if is_train and augmention:
            t = np.concatenate((np.random.permutation(folder_list), np.random.permutation(folder_list)))
        else:
            t = np.random.permutation(folder_list)
            
        if (len(t)%batch_size) == 0:
            num_batches = int(len(t)/batch_size)
        else:
            num_batches = len(t)//batch_size + 1
            
        # Iterate over the number of batches
        for batch in range(num_batches):

            # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,x,y,z,3))
            
            # batch_labels is the one hot representation of the output
            batch_labels = np.zeros((batch_size,5)) 

            # Iterate over the batch_size
            for folder in range(batch_size): 
                if debug:
                    plt.figure(figsize=(20,5))
                    
                # Handling remaining datapoints
                folder_idx = folder + (batch*batch_size)
                if folder_idx >= len(t):
                    break
                folder_str = t[folder_idx]
                imgs = os.listdir(source_path+'/'+ folder_str.split(';')[0]) # read all the images in the folder
                
                # Augmentation (randomly enabling)
                aug_type = None
                if is_train and augmention and rn.randint(0,1) == 1:
                    aug_type = rn.randint(0, 4) #randomly selecting augmentation type
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ folder_str.strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    # plotting original images for debugging purpose only
                    if debug:
                        plt.subplot(2, x, idx+1)
                        plt.imshow(image.astype('uint8'))

                    ## Cropping and resize of images.
                    # Images are of 2 different shape and the conv3D will throw error if the inputs in a batch have different shapes.
                    # Cropping to have the same aspect ratio
                    if image.shape[1] > image.shape[0]:
                        diff_px = image.shape[1] - image.shape[0]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[0]
                        image = image[:, crop_start:crop_end]
                    elif image.shape[0] > image.shape[1]:
                        diff_px = image.shape[0] - image.shape[1]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[1]
                        image = image[:, crop_start:crop_end]

                    resized_im = imresize(image, (y,z))

                    if aug_type is not None:
                        if aug_type == 0: # edge Enhancement
                            resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.EDGE_ENHANCE))
                        elif aug_type == 1: # adding gaussian blur
                            resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.GaussianBlur(1)))
                        elif aug_type == 2: # enchancing image detailing
                            resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.DETAIL))
                        elif aug_type == 3: # sharpening image
                            resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.SHARPEN))
                        elif aug_type == 4: # Brightness enhancement
                            resized_im = np.array(ImageEnhance.Brightness((Image.fromarray(resized_im, 'RGB'))).enhance(1.5))
                            
                    # plotting rezised images for debugging purpose only
                    if debug:
                        plt.subplot(2, x, idx+x+1)
                        plt.imshow(resized_im)

                    batch_data[folder,idx,:,:,0] = resized_im[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_im[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_im[:,:,2]/255 #normalise and feed in the image

                batch_labels[folder, int(folder_str.strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do            

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [24]:
curr_dt_time = datetime.datetime.now()
train_path = '_data/train'
val_path =  '_data/val'

#multiply number train seq by 2 when using augmentation
multiplier = 1
if enable_augmentation:
    multiplier = 2
num_train_sequences = len(train_doc)*multiplier
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [32]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

input_shape = (len(seq_idx), dim_x, dim_y, 3)
np.random.seed(30)

### Model 1 and 2 - Conv3D


Model 1 - Settings: 
* Batch Size = 128
* Ablation = 20
* Augmentation = False
* LR = 0.01
* Seq Length = 10
* Epoch = 20
* Dim = 120x120

Model 2 - Settings: 
...
* Batch Size = 32


In [ ]:
model_1 = Sequential()
model_1.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model_1.add(MaxPooling3D(pool_size=2))

model_1.add(Conv3D(64, kernel_size=3, activation='relu'))
model_1.add(MaxPooling3D(pool_size=2))

model_1.add(Flatten())
model_1.add(Dense(256, activation='relu'))
model_1.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 1
# Train Accuracy: 0.15
# Validation Accuracy: 0.15

## COMMENTS Results - Experiment 2
# Train Accuracy: 0.14
# Validation Accuracy: 0.21

### Model 3 - Conv3D
Model 3 - Settings:

* Added two Conv3D layer.

In [ ]:
model_3 = Sequential()

model_3.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model_3.add(Conv3D(64, kernel_size=3, activation='relu'))
model_3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_3.add(Conv3D(128, kernel_size=3, activation='relu'))
model_3.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_3.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model_3.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_3.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model_3.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model_3.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 3
# * Train Accuracy: 0.21
# * Validation Accuracy: 0.19

### Models 4 and 5 - Conv3D
Model 5 - Settings:

* Ablation = None
* Epoch = 50

In [ ]:
model_4 = Sequential()

model_4.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model_4.add(Conv3D(64, kernel_size=3, activation='relu'))
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))
model_4.add(BatchNormalization())

model_4.add(Conv3D(128, kernel_size=3, activation='relu'))
model_4.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_4.add(BatchNormalization())

model_4.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model_4.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_4.add(BatchNormalization())

model_4.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model_4.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model_4.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_4.add(BatchNormalization())

model_4.add(Flatten())
model_4.add(Dense(512, activation='relu'))
model_4.add(BatchNormalization())
model_4.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 4
# * Train Accuracy: 0.9053
# * Validation Accuracy: 0.2608

## COMMENTS Results - Experiment 5
# * Train Accuracy: 0.9172
# * Validation Accuracy: 0.71

### Models 6 and 7 - Conv3D

Model 6 - Settings:
* Drop out = 0.2

Model 7 - Settings:
* Drop out = 0.5

In [ ]:
model_6 = Sequential()

model_6.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model_6.add(Conv3D(64, kernel_size=3, activation='relu'))
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))
model_6.add(BatchNormalization())
#model_6.add(Dropout(0.2))
model_6.add(Dropout(0.5))

model_6.add(Conv3D(128, kernel_size=3, activation='relu'))
model_6.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_6.add(BatchNormalization())
#model_6.add(Dropout(0.2))
model_6.add(Dropout(0.5))

model_6.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model_6.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_6.add(BatchNormalization())
#model_6.add(Dropout(0.2))
model_6.add(Dropout(0.5))

model_6.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model_6.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model_6.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_6.add(BatchNormalization())
# model_6.add(Dropout(0.2))
model_6.add(Dropout(0.5))

model_6.add(Flatten())
model_6.add(Dense(512, activation='relu'))
model_6.add(BatchNormalization())
model_6.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 6
# * Train Accuracy: 0.9686
# * Validation Accuracy: 0.7523"

## COMMENTS Results - Experiment 7
# * Train Accuracy: 0.9671
# * Validation Accuracy: 0.5023"

### Model 8 - Conv3D

In [ ]:
model_8 = Sequential()

model_8.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model_8.add(Conv3D(64, kernel_size=3, activation='relu'))
model_8.add(MaxPooling3D(pool_size=(2, 2, 2)))
model_8.add(BatchNormalization())
model_8.add(Dropout(0.2))

model_8.add(Conv3D(128, kernel_size=3, activation='relu'))
model_8.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_8.add(BatchNormalization())
model_8.add(Dropout(0.2))

model_8.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model_8.add(MaxPooling3D(pool_size=(1, 2, 2)))
model_8.add(BatchNormalization())
model_8.add(Dropout(0.2))

model_8.add(GlobalAveragePooling3D())
model_8.add(Dense(512, activation='relu'))
model_8.add(BatchNormalization())
model_8.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 8
# * Train Accuracy: 0.9412
# * Validation Accuracy: 0.8021

### Model 9  - TimeDistributed Conv2D + GRU

In [ ]:
model_9 = Sequential()
model_9.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model_9.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_9.add(BatchNormalization())

model_9.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model_9.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_9.add(BatchNormalization())

model_9.add(TimeDistributed(GlobalAveragePooling2D()))
model_9.add(TimeDistributed(Dense(64, activation='relu')))
model_9.add(BatchNormalization())

model_9.add(GRU(128))
model_9.add(BatchNormalization())
model_9.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 9
# * Train Accuracy: 0.8680
# * Validation Accuracy: 0.8351

### Model 10 - TimeDistributed Conv2D + GRU
Model 10 - Settings:
* Drop out = 0.2

In [ ]:
model_10 = Sequential()
model_10.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model_10.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_10.add(BatchNormalization())
model_10.add(Dropout(0.2))

model_10.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model_10.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_10.add(BatchNormalization())
model_10.add(Dropout(0.2))

model_10.add(TimeDistributed(GlobalAveragePooling2D()))
model_10.add(TimeDistributed(Dense(64, activation='relu')))
model_10.add(BatchNormalization())
model_10.add(Dropout(0.2))

model_10.add(GRU(128))
model_10.add(BatchNormalization())
model_10.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 10
# * Train Accuracy: 0.8721
# * Validation Accuracy: 0.5611

### Model 11 - TimeDistributed Conv2D + Dense

In [ ]:
model_11 = Sequential()
model_11.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model_11.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_11.add(BatchNormalization())

model_11.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model_11.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_11.add(BatchNormalization())

model_11.add(TimeDistributed(
    Conv2D(128, (3,3), activation='relu'))
)
model_11.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model_11.add(BatchNormalization())

model_11.add(GlobalAveragePooling3D())
model_11.add(Dense(256, activation='relu'))
model_11.add(BatchNormalization())
model_11.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 11
# * Train Accuracy: 0.8670
# * Validation Accuracy: 0.8450

## FINAL MODEL - Time Distributed + ConvLSTM2D

In [33]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(8, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Conv2D(16, (3,3), activation='relu'))
)
model.add(BatchNormalization())
model.add(
    ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Dense(64, activation='relu'))
)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

## COMMENTS Results - Experiment 12
# * Train Accuracy: 0.9673
# * Validation Accuracy: 0.9375

2022-05-18 02:14:30.576314: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 02:14:30.577239: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [34]:
optimiser = optimizers.Adam(lr=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 10, 118, 118, 8)   224       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 118, 118, 8)   32        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 116, 116, 16)  1168      
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 116, 116, 16)  64        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 114, 114, 8)       6944      
_________________________________________________________________
batch_normalization_3 (Batch (None, 114, 114, 8)       32        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 114, 114, 64)     

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [37]:
train_generator = generator(train_path, train_doc, batch_size, is_train = True, augmention = enable_augmentation)
val_generator = generator(val_path, val_doc, batch_size)

In [38]:
model_name = 'model_init_exp_16' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [39]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                        callbacks=callbacks_list, validation_data=val_generator, 
                        validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 279s 13s/step - loss: 0.6895 - categorical_accuracy: 0.7247 - val_loss: 1.4734 - val_categorical_accuracy: 0.5859

Epoch 00001: saving model to model_init_exp_16_2022-05-1802_11_54.319839/model-00001-0.68946-0.72470-1.47341-0.58594.h5
Epoch 2/50
21/21 [==============================] - 339s 16s/step - loss: 0.6785 - categorical_accuracy: 0.7039 - val_loss: 1.0108 - val_categorical_accuracy: 0.6953

Epoch 00002: saving model to model_init_exp_16_2022-05-1802_11_54.319839/model-00002-0.67853-0.70387-1.01078-0.69531.h5
Epoch 3/50
21/21 [==============================] - 319s 15s/step - loss: 0.6794 - categorical_accuracy: 0.7336 - val_loss: 0.7951 - val_categorical_accuracy: 0.7266

Epoch 00003: saving model to model_init_exp_16_2022-05-1802_11_54.319839/model-00003-0.67943-0.73363-0.79512-0.72656.h5
Epoch 4/50
21/21 [==============================] - 276s 13s/step - loss: 0.6653 - categorical_accuracy: 0.7202 - val_loss: 1.1786 - val_ca


Epoch 00028: saving model to model_init_exp_16_2022-05-1802_11_54.319839/model-00028-0.61861-0.71875-1.46743-0.67969.h5
Epoch 29/50
21/21 [==============================] - 279s 13s/step - loss: 0.6536 - categorical_accuracy: 0.7173 - val_loss: 1.3548 - val_categorical_accuracy: 0.6172

Epoch 00029: saving model to model_init_exp_16_2022-05-1802_11_54.319839/model-00029-0.65365-0.71726-1.35480-0.61719.h5
Epoch 30/50
21/21 [==============================] - 283s 13s/step - loss: 0.6706 - categorical_accuracy: 0.7113 - val_loss: 1.1341 - val_categorical_accuracy: 0.6953

Epoch 00030: saving model to model_init_exp_16_2022-05-1802_11_54.319839/model-00030-0.67058-0.71131-1.13412-0.69531.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 31/50
21/21 [==============================] - 275s 13s/step - loss: 0.6681 - categorical_accuracy: 0.7366 - val_loss: 1.2008 - val_categorical_accuracy: 0.6094

Epoch 00031: saving model to model_init_exp_16_2022-05-1802_11_54.3198

In [ ]:
plt.figure(figsize=(20,6))
ax1 = plt.subplot(121)
ax1 = plt.plot(history.history['loss'])
ax1 = plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')
ax2 = plt.subplot(122)
ax2 = plt.plot(history.history['categorical_accuracy'])
ax2 = plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')